## ASSIGNMENT-2

Roll Numbers: 2023121013, 2023122002, 2023121006

Names: Himanshu Singh, Srinath Bhamidipati, Pavan Karke

### Instructions
 * Fill in the roll numbers and names of all the teams members in the cell above.
 * Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
 * All the code and result files should be uploaded in the github classroom.
 * You can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
 * All the representations are expected to be in a right-hand coordinate system. All the functions related to transformation matrices, quaternions, and 3D projection are expected to be coded by you.
 * You could split the Jupyter Notebook cells where TODO is written, but please try to avoid splitting/changing the structure of other cells.
 * All the visualization should be done inside the notebook unless specified otherwise.
 * Plagiarism will lead to heavy penalty.

## G2O Motion Model 

Using the following motion model, you have to first generate the "initialization" for all the poses/vertices using the "Given" information. Just like in the 1D case.
$$x_{k+1} = x_{k} + \Delta x_{(k,k+1)} \cos(\theta_k) - \Delta y_{(k,k+1)} \sin(\theta_k) \\
y_{k+1} = y_{k} + \Delta y_{(k,k+1)} \cos(\theta_k) + \Delta x_{(k,k+1)} \sin(\theta_k) \\
\theta_{k+1} = \theta_{k}+  \Delta \theta_{(k,k+1)}$$

Even the loop closure nodes are related by the above model, except that it need not necessarily be consecutive nodes k and k+1.

## PART 2

In [1]:
# Imports

import math
import os

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def apply_transformation(points, transformation):
    """ Applies a 4x4 transformation matrix to an array of points. """

    rotation = transformation[:3, :3]
    translation = transformation[:3, 3]
    transformed_points = (points @ rotation.T) + translation
    return transformed_points

### 1. Aligning Point Clouds using ICP (10 marks)

Given two 3D point clouds, P (`original_point_cloud.ply`) and Q (`transformed_point_cloud.ply`), where the second point cloud Q is a transformed version of the first point cloud P, apply the Iterative Closest Point (ICP) algorithm to align the two point clouds

Implement ICP using the Levenberg-Marquardt algorithm discussed in class. Compute the registration error and visualize the quality of alignment. 

Compare these results with the earlier ICP with SVD. Write a brief explanation on which one performs better (both time wise and accuracy wise), and why you think that is.

Correspondences are {P_i, Q_i} {i =  1 to N}

In [3]:
def icp_svd(P, Q):
    """ Performs Procrustes alignment on two point clouds
    and returns the transformation matrix."""

    # Compute the mean of the point clouds
    P_mean = np.mean(P, axis=0)
    Q_mean = np.mean(Q, axis=0)

    # Center the two point clouds around their means
    P_centered = P - P_mean
    Q_centered = Q - Q_mean

    # Compute the cross-covariance matrix
    W = Q_centered.T @ P_centered

    # Compute the SVD of the cross-covariance matrix
    U, S, V_t = np.linalg.svd(W)

    # Compute the rotation and translation
    R = U @ V_t
    T = Q_mean - (R @ P_mean)

    # Construct the transformation matrix
    transformation = np.vstack((np.hstack((R, T.reshape(-1, 1))), [0, 0, 0, 1]))

    return transformation

# Read the point cloud
PC_P = o3d.io.read_point_cloud("data/original_point_cloud.ply")
P = np.asarray(PC_P.points)

# Read the transformed point cloud
PC_Q = o3d.io.read_point_cloud("data/transformed_point_cloud.ply")
Q = np.asarray(PC_Q.points)

# Predict the applied transformation
T_pred = icp_svd(P, Q)

# Apply the predicted transformation
Q_pred = apply_transformation(P, T_pred)

# Predicted point cloud
PC_Q_pred = o3d.geometry.PointCloud()
PC_Q_pred.points = o3d.utility.Vector3dVector(Q_pred)

# Compute RMSE between the two transformed point clouds
alignment_error = np.sqrt(np.mean((Q - Q_pred) ** 2))
print('Alignment Error:', alignment_error)

# Visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
# vis.add_geometry(PC_P)
vis.add_geometry(PC_Q)
vis.add_geometry(PC_Q_pred)
vis.poll_events()
vis.update_renderer()
vis.run()
vis.destroy_window()

Alignment Error: 1.7728025322685544e-07


In [4]:
def skew_symmetric(v):
    """ Convert a vector to a skew-symmetric matrix. """

    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def exponential_map(xi):
    """ Exponential map from the local tangent space to the group. """

    # Break into components
    omega = xi[:3]
    v = xi[3:]

    # Compute rotation matrix
    theta = np.linalg.norm(omega)
    omega_hat = skew_symmetric(omega)
    R = np.eye(3)
    R += (np.sin(theta) / theta) * omega_hat
    R += ((1 - np.cos(theta)) / (theta * theta)) * (omega_hat @ omega_hat)

    # Compute transformation matrirx
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = R @ v
    return T

def icp_levenberg_marquardt(P, Q, max_iterations=50, tolerance=1e-8):
    """ Given a transformation matrix, transforms toothless.ply
    and recovers the transformation using Lie Group Optimization."""

    # Number of points
    assert P.shape == Q.shape
    num_points = P.shape[0]

    # Predict the applied transformation
    T_pred = np.eye(4)

    # Iterate until max iterations
    for iteration in range(max_iterations):

        # Apply the current transformation
        Q_pred = apply_transformation(P, T_pred)

        # Compute the residuals
        residuals = Q - Q_pred

        # Compute the Jacobian matrix
        J = np.zeros((3 * num_points, 6))
        for idx in range(num_points):
            J[3*idx:3*idx+3, :3] = -skew_symmetric(Q_pred[idx])
            J[3*idx:3*idx+3, 3:] = np.eye(3)

        # Compute the update using Gauss-Newton method
        H = J.T @ J
        g = J.T @ residuals.flatten()
        delta_xi = np.linalg.solve(H, g)

        # Update the transformation
        delta_T = exponential_map(delta_xi)
        T_pred = delta_T @ T_pred

        # Check for convergence
        if np.linalg.norm(delta_xi) < tolerance:
            break

    return T_pred

# Read the point cloud
PC_P = o3d.io.read_point_cloud("data/original_point_cloud.ply")
P = np.asarray(PC_P.points)

# Read the transformed point cloud
PC_Q = o3d.io.read_point_cloud("data/transformed_point_cloud.ply")
Q = np.asarray(PC_Q.points)

# Predict the applied transformation
T_pred = icp_svd(P, Q)

# Apply the predicted transformation
Q_pred = apply_transformation(P, T_pred)

# Predicted point cloud
PC_Q_pred = o3d.geometry.PointCloud()
PC_Q_pred.points = o3d.utility.Vector3dVector(Q_pred)

# Compute RMSE between the two transformed point clouds
alignment_error = np.sqrt(np.mean((Q - Q_pred) ** 2))
print('Alignment Error:', alignment_error)

# Visualization
vis = o3d.visualization.Visualizer()
vis.create_window()
# vis.add_geometry(PC_P)
vis.add_geometry(PC_Q)
vis.add_geometry(PC_Q_pred)
vis.poll_events()
vis.update_renderer()
vis.run()
vis.destroy_window()

Alignment Error: 1.7728025322685544e-07


### 2. LieTorch (15 marks)
Resources: https://github.com/princeton-vl/lietorch/tree/master

The aim of this question is to use LieTorch for Pose Graph Optimization. You will be given a set of `n` noisy Transforms (representing your poses) and a set of `m` Loop Closure edges which will be generated by a function we have implemented. There are 2 parts for this question:
1. Keep the loop closure edges constant but adjust the `n` adjacent Transforms
2. Find a way to integrate confidence into the optimization (1 confidence value for the loop closure edges, and another for the adjacent edges), so that both the loop closure edges and the adjacent edges are optimized but to different degrees based on the confidence scores (Higher confidence implies that it's more likely that your edge is correct).

**Note:** Document your loss functions along with your implementation, with suitable explanation for the design choices you make. Also print your loss values, to show them converging.

Plot the original X,Y,Z values of your pose, their noisy counterparts and the new X,Y,Z values after optimization.

In [3]:
def getTransforms():
    '''
    Output Format - n, m, [i, j, x, y, z, q_x, q_y, q_z, q_w] * (m+n)
    The first n values will be the adjacent edges
    The last m values will be the loop closure edges

    '''
    pass


#### How you can mimic the getTransforms function:
Generate n random Transformations, create the loop closure edges between m of them at random, and then add noise to your original n Transformations.


In [ ]:
'''
YOUR CODE HERE
'''

### 3. Bonus (10 marks)

Implement the Pose Graph Optimization for the same G2O file as before using LieTorch, and compare the results against G2O using the Evo metrics.

You are not allowed to call any external python files for this question

Why is it performing better/worse?